In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV , RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import uniform
import os
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [2]:
# List files in the directory
file_list = os.listdir('/kaggle/input/uber-car-booking')
print("Files in the directory:", file_list)

Files in the directory: ['final_internship_data.csv']


In [3]:
# Specify the correct file path
file_path = '/kaggle/input/uber-car-booking/final_internship_data.csv'
df = pd.read_csv(file_path)

In [4]:
print("First few rows of the dataset:")
df.head()

First few rows of the dataset:


,User ID,User Name,Driver Name,Car Condition,Weather,Traffic Condition,key,fare_amount,pickup_datetime,pickup_longitude,...,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
0,KHVrEVlD,Kimberly Adams,Amy Butler,Very Good,windy,Congested Traffic,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-1.288826,...,6,0,2009,20.265840,55.176046,14.342611,34.543548,27.572573,1.030764,-2.918897
1,lPxIuEri,Justin Tapia,Hannah Zimmerman,Excellent,cloudy,Flow Traffic,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-1.291824,...,1,1,2010,44.667679,31.832358,23.130775,15.125872,8.755732,8.450134,-0.375217
2,gsVN8JLS,Elizabeth Lopez,Amanda Jackson,Bad,stormy,Congested Traffic,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-1.291242,...,8,3,2011,43.597686,33.712082,19.865289,17.722624,9.847344,1.389525,2.599961
3,9I7kWFgd,Steven Wilson,Amy Horn,Very Good,stormy,Flow Traffic,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-1.291319,...,4,5,2012,42.642965,32.556289,21.063132,15.738963,7.703421,2.799270,0.133905
4,8QN5ZaGN,Alexander Andrews,Cassandra Larson,Bad,stormy,Congested Traffic,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-1.290987,...,3,1,2010,43.329953,39.406828,15.219339,23.732406,15.600745,1.999157,-0.502703


In [5]:
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

Number of rows: 500000
Number of columns: 26


In [6]:
print("\nBasic information about the dataset:")
df.info()


Basic information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   User ID            500000 non-null  object 
 1   User Name          500000 non-null  object 
 2   Driver Name        500000 non-null  object 
 3   Car Condition      500000 non-null  object 
 4   Weather            500000 non-null  object 
 5   Traffic Condition  500000 non-null  object 
 6   key                500000 non-null  object 
 7   fare_amount        500000 non-null  float64
 8   pickup_datetime    500000 non-null  object 
 9   pickup_longitude   500000 non-null  float64
 10  pickup_latitude    500000 non-null  float64
 11  dropoff_longitude  499995 non-null  float64
 12  dropoff_latitude   499995 non-null  float64
 13  passenger_count    500000 non-null  int64  
 14  hour               500000 non-null  int64  
 15  day          

In [7]:
print("\nSummary statistics for integer columns:")
df.describe()


Summary statistics for integer columns:


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day,month,weekday,year,jfk_dist,ewr_dist,lga_dist,sol_dist,nyc_dist,distance,bearing
count,500000.000000,500000.000000,500000.000000,499995.000000,499995.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,499995.000000,499995.000000,499995.000000,499995.000000,499995.000000,499995.000000,499995.000000
mean,11.358361,-1.265712,0.696740,-1.265755,0.696675,1.683428,13.510834,15.684206,6.268650,3.042008,2011.739132,385.279367,380.503657,363.843772,363.674038,355.991423,19.468775,0.297145
std,9.916617,0.206941,0.140909,0.205903,0.128997,1.307395,6.511571,8.681066,3.437815,1.949240,1.860889,2419.087483,2428.804740,2425.075903,2428.348683,2428.730839,367.299601,1.804548
min,-44.900000,-52.119764,-54.389440,-59.049665,-44.676047,0.000000,0.000000,1.000000,1.000000,0.000000,2009.000000,1.017646,1.460945,0.382119,0.532545,0.080500,0.000000,-3.141593
25%,6.000000,-1.291405,0.710958,-1.291393,0.710943,1.000000,9.000000,8.000000,3.000000,1.000000,2010.000000,41.341514,32.173712,17.100762,14.886989,7.147384,1.214550,-0.854721
50%,8.500000,-1.291226,0.711268,-1.291197,0.711277,1.000000,14.000000,16.000000,6.000000,3.000000,2012.000000,42.523163,34.787507,19.591554,18.347580,10.458151,2.116970,-0.050442
75%,12.500000,-1.290970,0.711520,-1.290908,0.711538,2.000000,19.000000,23.000000,9.000000,5.000000,2013.000000,43.785649,38.304502,22.214815,22.417812,14.448699,3.890070,2.206769
max,500.000000,37.360538,29.724576,0.712985,7.061893,6.000000,23.000000,31.000000,12.000000,6.000000,2015.000000,30133.067880,30167.595967,30167.285794,30159.407296,30162.285356,12399.956433,3.141593


In [8]:
print("\nMissing values in the dataset:")
print(df.isnull().sum())


Missing values in the dataset:
User ID              0
User Name            0
Driver Name          0
Car Condition        0
Weather              0
Traffic Condition    0
key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    5
dropoff_latitude     5
passenger_count      0
hour                 0
day                  0
month                0
weekday              0
year                 0
jfk_dist             5
ewr_dist             5
lga_dist             5
sol_dist             5
nyc_dist             5
distance             5
bearing              5
dtype: int64


In [9]:
# Drop irrelevant columns
df = df.drop(['User ID', 'User Name', 'Driver Name', 'key', 'pickup_datetime'], axis=1)

In [10]:
# Handle missing values
df = df.dropna()

In [11]:
# Remove outliers using IQR (Interquartile Range)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [12]:
# Apply outlier removal to numerical columns
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_columns:
    df = remove_outliers(df, col)

In [13]:
# Encode categorical variables
df = pd.get_dummies(df, columns=['Car Condition', 'Weather', 'Traffic Condition'], drop_first=True)

In [14]:
# Define features and target
X = df.drop('fare_amount', axis=1)
y = df['fare_amount']

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [18]:
# Evaluate Models
def evaluate_model(y_true, y_pred, model_name, tolerance=0.05):
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Calculate MAE
    mae = mean_absolute_error(y_true, y_pred)
    # Calculate R²
    r2 = r2_score(y_true, y_pred)
    # Calculate tolerance-based accuracy
    within_tolerance = np.abs((y_true - y_pred) / y_true) <= tolerance
    accuracy = np.mean(within_tolerance) * 100  # Convert to percentage
    # Print the results
    print(f"{model_name} - RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.2f}, Accuracy (±{tolerance*100:.0f}%): {accuracy:.2f}%")

In [19]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsRegressor(n_neighbors=3)  # Default hyperparameters
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)

# Evaluate KNN
evaluate_model(y_test, y_pred_knn, "K-Nearest Neighbors (KNN)")

K-Nearest Neighbors (KNN) - RMSE: 2.50, MAE: 1.85, R²: 0.47, Accuracy (±5%): 14.43%


In [18]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsRegressor(n_neighbors=5)  # Default hyperparameters
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)

# Evaluate KNN
evaluate_model(y_test, y_pred_knn, "K-Nearest Neighbors (KNN)")

K-Nearest Neighbors (KNN) - RMSE: 2.37, MAE: 1.75, R²: 0.53, Accuracy (±5%): 15.47%


In [20]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsRegressor(n_neighbors=10)  # Default hyperparameters
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)

# Evaluate KNN
evaluate_model(y_test, y_pred_knn, "K-Nearest Neighbors (KNN)")

K-Nearest Neighbors (KNN) - RMSE: 2.28, MAE: 1.69, R²: 0.56, Accuracy (±5%): 15.72%


In [19]:
# K-Nearest Neighbors (KNN)
knn = KNeighborsRegressor(n_neighbors=20)  # Default hyperparameters
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)

# Evaluate KNN
evaluate_model(y_test, y_pred_knn, "K-Nearest Neighbors (KNN)")

K-Nearest Neighbors (KNN) - RMSE: 2.26, MAE: 1.67, R²: 0.57, Accuracy (±5%): 15.85%


In [19]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train_pca, y_train)
y_pred_lr = lr.predict(X_test_pca)
# Evaluate
evaluate_model(y_test, y_pred_lr, "Linear Regression")

Linear Regression - RMSE: 2.14, MAE: 1.53, R²: 0.61, Accuracy (±5%): 17.22%


In [20]:
# Polynomial Regression with Linear Regression
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_pca)
X_test_poly = poly.transform(X_test_pca)

lr_poly = LinearRegression()
lr_poly.fit(X_train_poly, y_train)
y_pred_poly = lr_poly.predict(X_test_poly)

evaluate_model(y_test, y_pred_poly, "Polynomial Regression")

Polynomial Regression - RMSE: 2.00, MAE: 1.40, R²: 0.66, Accuracy (±5%): 19.12%


In [22]:
# Random Forest
rf = RandomForestRegressor(n_estimators=20, max_depth=None, random_state=42)  # Default hyperparameters
rf.fit(X_train_pca, y_train)
y_pred_rf = rf.predict(X_test_pca)

# Evaluate Random Forest
evaluate_model(y_test, y_pred_rf, "Random Forest")

Random Forest - RMSE: 1.99, MAE: 1.42, R²: 0.67, Accuracy (±5%): 18.88%


In [21]:
# Random Forest
rf = RandomForestRegressor(n_estimators=50, max_depth=None, random_state=42)  # Default hyperparameters
rf.fit(X_train_pca, y_train)
y_pred_rf = rf.predict(X_test_pca)

# Evaluate Random Forest
evaluate_model(y_test, y_pred_rf, "Random Forest")

Random Forest - RMSE: 1.95, MAE: 1.39, R²: 0.68, Accuracy (±5%): 19.22%


In [21]:
# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=42)  # Default hyperparameters
rf.fit(X_train_pca, y_train)
y_pred_rf = rf.predict(X_test_pca)

# Evaluate Random Forest
evaluate_model(y_test, y_pred_rf, "Random Forest")

Random Forest - RMSE: 1.94, MAE: 1.38, R²: 0.68, Accuracy (±5%): 19.39%


In [22]:
# Fine-tuned Random Forest
rf_fine_tuned = RandomForestRegressor(n_estimators=200,       # Increase the number of trees
    max_depth=20,           # Limit the depth of each tree
    min_samples_split=5,    # Require more samples to split a node
    min_samples_leaf=2,     # Require more samples to be at a leaf node
    max_features='sqrt',    # Use sqrt(n_features) for splitting
    random_state=42 )
# Train the fine-tuned model
rf_fine_tuned.fit(X_train_pca, y_train)
# Predict on the test set
y_pred_rf_fine_tuned = rf_fine_tuned.predict(X_test_pca)
# Evaluate the fine-tuned model
evaluate_model(y_test, y_pred_rf_fine_tuned, "Random Forest (Fine-Tuned)")

Random Forest (Fine-Tuned) - RMSE: 1.95, MAE: 1.39, R²: 0.68, Accuracy (±5%): 19.00%


In [25]:
# Further fine-tuned Random Forest
rf_fine_tuned = RandomForestRegressor(
    n_estimators=300,       # More trees for better stability
    max_depth=25,           # Allow deeper trees
    min_samples_split=3,    # Allow more splits for better learning
    min_samples_leaf=1,     # Leaf nodes with single samples to capture fine details
    max_features='auto',    # Use all features for better prediction power
    bootstrap=False,        # Train on the full dataset for stronger models
    random_state=42,
    n_jobs=-1               # Use all CPU cores for faster training
)
# Train the fine-tuned model
rf_fine_tuned.fit(X_train_pca, y_train)
# Predict on the test set
y_pred_rf_fine_tuned = rf_fine_tuned.predict(X_test_pca)
# Evaluate the improved model
evaluate_model(y_test, y_pred_rf_fine_tuned, "Random Forest (Optimized)")

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Random Forest (Optimized) - RMSE: 2.68, MAE: 1.89, R²: 0.39, Accuracy (±5%): 15.44%


In [23]:
# Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)  # Default hyperparameters
gb.fit(X_train_pca, y_train)
y_pred_gb = gb.predict(X_test_pca)
evaluate_model(y_test, y_pred_gb, "Gradient Boosting")

Gradient Boosting - RMSE: 2.03, MAE: 1.45, R²: 0.65, Accuracy (±5%): 18.46%


In [24]:
# XGBoost
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)  # Default hyperparameters
xgb.fit(X_train_pca, y_train)
y_pred_xgb = xgb.predict(X_test_pca)
evaluate_model(y_test, y_pred_xgb, "XGBoost")

XGBoost - RMSE: 2.03, MAE: 1.45, R²: 0.65, Accuracy (±5%): 18.47%
